In [83]:
import numpy as np
import re
import itertools
from collections import Counter
from tensorflow.contrib import learn

# Funcoes Auxiliares (tiradas do paper)

In [52]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

# Data explore

In [75]:
import random

data,labels = load_data_and_labels('data/rt-polaritydata/rt-polarity.pos','data/rt-polaritydata/rt-polarity.neg')

print (len(data)) #todos os reviews, cada elemento eh um review
print (len(labels)) #todos os sentimentos


10662
10662


In [76]:
(data[0], labels[0]) #tupla primeiro review, no caso eh positivo

("the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal",
 array([0, 1]))

# Pre processing
- Build vocabulary

In [131]:
max_doc_len = max([len(x.split(" ")) for x in data])

#Maps documents to sequences of word ids.
#Basicamente cada palavra ja eh mapeada para um id e a sentenca tb
vocab_processor = learn.preprocessing.VocabularyProcessor(max_doc_len)
x = np.array(list(vocab_processor.fit_transform(data)))#aqui que ocorre o mapeamento
(x[0], data[0])

(array([ 1,  2,  3,  4,  5,  6,  1,  7,  8,  9, 10, 11, 12, 13, 14,  9, 15,
         5, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]),
 "the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal")

- Randomly shuffle data

In [134]:
#Shuffle Data
np.random.seed(10) #for debugging, garante que os numeros aleatorios gerados sempre sejam os mesmos
shuffle_indices = np.random.permutation(np.arange(len(labels)))
shuffle_indices

array([ 7359,  5573, 10180, ...,  1344,  7293,  1289])

In [135]:
# basicamente eu refaço o array de arrays, mas agora cada indice eh oq ta como indice gerado do random
#ex: x[7359] == x_shuffled[0] assim os dois (data, label) mantem o mesmo indice
x_shuffled = x[shuffle_indices] #aqui sao as sentencas ja mapeadas
y_shuffled = labels[shuffle_indices]



 - Train/Validation split

In [136]:
val_percentage = .1
val_sample_index = -1 * int(val_percentage * float(len(labels)))
#justamente pra pegar os ultimos enquanto no validation, pega de ordem reversa,
#ex: pegue todos com excessao dos -1066 ultimos
#ou pegue os 1066 primeiros
#TODO - fazer do meu jeito isso aqui

In [137]:
x_train, x_val = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

In [138]:
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 18758
Train/Dev split: 9596/1066


56